# Notes on Statistical Inferencing

### Probability
* __Multivariate Gaussian density__ (this is similar to the exponential of a single variable gaussian PDF)
    <blockquote>$$
    p(\vec{x}) \propto \exp \left [ - \frac{1}{2} (\vec{x} -
        \vec{\mu})^\mathrm{T} \, \Sigma ^{-1} \, (\vec{x} - \vec{\mu})
        \right ]$$

    where $\vec{\mu}$ is an $N$-dimensional vector position of the mean of the density and $\Sigma$ is the square N-by-N covariance matrix.
    </blockquote>
* __covariance:__ measure of the relationship between two random variables
    * $\sigma(x,y)= \frac{1}{n-1}\sum_{i=1}^{n}(x_i-\bar{x})(y_i-\bar{y})$
* __variance:__ measure of the spread of a sample/population
    * $S^2 = \sum(x_i - \bar{x}) \ / \ n-1$ (for sample)
    * Also, the variance can be described as the covariance of a single variable $\sigma(x, x)$


### Ways to Interpolate data
* Most basic MCMC: the __Metropolis-Hastings__ algorithm
    * One chain, one walker
    * Likelihood function, priors, and proposal distribution (walker)
        * Data set has certain mean, variance, standard deviation
        * The walker finds the values in parameter space to probe (randomly)
        * Feeds the points to likelihood function (usually gaussian) and priors
            * Distribution of a likelihood function and priors
            * These are multiplied together
    * [More on Metropolis-Hastings](https://github.com/Joseph94m/MCMC/blob/master/MCMC.ipynb)
* __Linear Least squares:__ finding a line that best fits several linear equations
    * $A\vec{x}=b$ has least squares solutions $A^{T}A\vec{x}=A^Tb$ (normal equation)
    * The least squares solution to $A\vec{x}=b$ is $x^{*}=(A^TA)^{-1}A^Tb$
    * The function takes the form: $f_{\rm{lsfit}}(s) = x_1f_1(s) + ... + x_nf_n(s)$
    * __Linear Least Squares Polynomial Fitting:__ a way of finding a polynomial that best fits several coordinates
        * Requires __vandermonde matrix__ 
            * Matrix takes the form: $$\begin{equation*}
    A = \begin{pmatrix}
      1 & x_0 & x_0^2 & \cdots & x_0^{n-1} &
      x_0^{n}\\
      1 & x_1 & x_1^2 & \cdots & x_1^{n-1} &
      x_1^{n}\\
      \vdots & \vdots  & \vdots & \ddots & \vdots & \vdots\\
      1 & x_n & x_n^2 & \cdots & x_n^{n-1} &
      x_n^{n}
    \end{pmatrix},
    \end{equation*}$$
            * Function takes the form:
            $$(a_{1}x^{n-1}+a_{2}x^{n-2}... + a_{n-1}x+a_{n}) = y$$
            * Input: x, y points
            * Output: fitted polynomial
            * __How to (basic)__:
                * Points: (0, 1), (1, 0), (2/3, 1/2) ($n$ pairs of coordinates)
                * Each row of the matrix takes the form: $(a_{1}x^{n-1}+a_{2}x^{n-2}... + a_{n-1}x+a_{n}) = y$
                    * EX: First pair of coordinates would be: $(a_{1}0^{2}+a_{2}0^{1}+a_{3}0^0) =1$
                    * Forms augmented matrix:
                        * $\begin{pmatrix} 1&0&0&1\\ 1&1&1&0\\ 1&\frac{2}{3}&\frac{4}{9}&\frac{1}{2}\end{pmatrix} \ $ or, more familiarly,  $\ \begin{pmatrix}1&0&0\\ 1&1&1\\ 1&\frac{2}{3}&\frac{4}{9}\end{pmatrix}\begin{pmatrix}a_0\\ a_1\\ a_2\end{pmatrix}=\begin{pmatrix}1\\ 0\\ \frac{1}{2}\end{pmatrix}$
                    * Solution: $= -\frac{3}{4}x^2-\frac{1}{4}x+1$
    * Code (from [`emcee` notebook](https://emcee.readthedocs.io/en/stable/tutorials/line/)):
```python
A = np.vander(x, 2) # make vandermonde matrices (shape 50, 2) out of x values
C = np.diag(yerr * yerr) # make diagonal matrix out of square of y-errors
ATA = np.dot(A.T, A / (yerr ** 2)[:, None]) #finds A^T•A and divides by error squared (C) shape(2,2)
cov = np.linalg.inv(ATA) #inverse
w = np.linalg.solve(ATA, np.dot(A.T, y / yerr ** 2)) #this is least-squares solution
print("Least-squares estimates:")
print("m = {0:.3f} ± {1:.3f}".format(w[0], np.sqrt(cov[0, 0]))) #error is sqrt of diagonal of covariance matrix
print("b = {0:.3f} ± {1:.3f}".format(w[1], np.sqrt(cov[1, 1])))
```
    * In math notation:
        * $AX=Y$ with solution $w=x^*=[A^T C^{-1} A]^{-1}[A^TC^{-1}Y]$
            * $A$ = vandermonde matrix
            * $C^{-1}$ = divided by __covariance matrix__: $C(m, b) = \begin{pmatrix}\sigma_{b}^2&\sigma_{mb}^2\\ \sigma_{mb}^2&\sigma_{m}^2\end{pmatrix}$
                * in the code, $C$ is determined by the diagonal of the error in y
            * $Y$ = distribution of y values (with error and added fractional uncertainty)



### 9/14 Meeting Notes

* Dr. Andrews/Teague have been working on statistical inferencing of the spectral line model
    * __Statistical inferencing:__ statistical techniques (in this case, Bayesian analysis) that tell us how well the model measures individual parameters ($m_{*}$, $\rm{T}_b$, $\Delta v$, etc.)
        * Posterior Probability- Bayesian probability of the accuracy of the models given data/outside information
            * __Bayes Theorem:__ $$\Pr ( M \mid D ) = \Pr(D \mid M) * \Pr(M) \ / \Pr(D)$$
            * _In words: The probability of the model given the data is equal to the probability of the data given the model (__likelihood__) multiplied by the probability of the data (__prior__) divided by the probability of the data (__Bayesian evidence__)_
                * __likelihood:__ ($\rm{L}$) The probability of the model given the data
                    * statistical inferencing is done in natural log because it is assumed that probabilities are gaussian, and the natural log of a gaussian is ~ roughly ~ the area under the curve
                    * Found through __chi-squared analysis__
                        * $\chi^2=(D-M)^2 \ / \ (\rm{noise})$
                            * roughly, subtract model channel maps from data channel maps
                        * $\ln(\rm{L}) = \Sigma_{i=0} = -\chi^2 \ / \ 2$ (__double check__)
                            * sum over visibilities (for now, all channel maps for data and model)
                * __prior:__ $(\Pr(M))$ the assumptions we make about the data
                    * assumptions like the range of the data (e.g. $0-1000 \rm{K}$), the gaussian distribution of the data (e.g. $370 \pm 10 \ \rm{K}$)
                * __Bayesian evidence:__ $(\Pr(D))$ hard to measure, but not necessary
                    * __MCMC algorithms__ do not need this quantity to work
        * In reality, $\Pr ( M \mid D ) \sim \Pr(D \mid M) * \Pr(M) \ / \Pr(D)$
            * Because $\Pr(D)$ is hard to measure, we use __Markov Chain Monte Carlo (MCMC)__ to weigh this bayesian probability equation in relation to proportions of the numerator, not exact values
            * __Markov Chain Monte Carlo__  uses a __Markov chain__ to sample probability at random points (random walks)
                * A __Monte Carlo__ algorithm is a way of randomly sampling a distribution to estimate the distributions of parameters given a set of observations
                * A __Markov chain__ looks at the ratio of probabilities between current chain and last chain to determine where to sample probability next
                    * More specifically, the next sample is determined by the correlation between the area of the PDF and the ratio
                        * Algorithm finds ratios that contribute significantly to the area of the PDF
                * A variant of MCMC (that will most likely be the algorithm used on models) is the __affine invariant ensemble sampler__ (`emcee` package)
                    * Essentially, multiple Markov chains occuring at once for faster processing
                        * Can specify number of chains

### 9/21 Notes
* Code that provides a MCMC output for 'fake data' `analyze_fit_emcee.py`
    * Goal: do we recover what we put in? We know the truth for the parameters, but does emcee handle everything correctly?
    * Fake data is generated with _no noise_ (no noise meaning that the typical noise from an interferometer is not accounted for (in the form of gaussians), only vague uncertainties
    * Code is slow on two accounts:
        * Making a cube to store data
            * __broadcasting__ arrays: handling arrays of different sizes for computation (NumPy)
                * i.e. matrix * scalar number (larger than 1)
            * computationally taxing
        * Post processing/interpolation for 1e6 different arrays for the likelihood calculation
        * Is possible to make it faster using Numba
            * Basically takes python code and makes it faster—cluster/machine optimized
    
* __Next steps:__
    * Testing a real dataset
        * Advantage: figure out what the errors are, sooner
        * Disadvantage: we don't know 'the truth'/what we're looking for, so we are subject to lots of biases
    * Technical biases
        * Training data to conform to the truth
        * make certain calibrations to improve computational speed
            * time-averaging: taking larger samples in time (i.e not ever 6 seconds, but every 30 seconds)
                * Question: Does it make sense to time-average? Do we still retain the same amount of information? __how to optimize__

* __Long term:__
    * Is it best to find out ideal conditions to collect interferometer data to probe for disk masses?
    * Or, is it best to figure out what a model of dynamical masses actually looks like and the errors associated with it?
    * Exploring both simultaneously (Teague, Andrews, and myself)

* __Miscellaneous:__
    * Spectral resolution: frequency/velocity at which you take measurements of the sky with interferometry
        * One technique to increase resolution: VLBI (Very-Long Baseline Interferometry)
    * Optimum data collection: higher spectral resolution
        * Caveats:
            * Telescope time is competitive
                * Observations fall into two camps:
                    * Snapshot (1 sec-1 min of sky)
                    * Medium length (20-30 minutes)
                    * Long (1-5 hrs)
            * Higher spectral resolution = higher noise (similar to fourier transform analysis)
    * Spectral continuum = 1 channel map



### Notes on `logL_test.py` (likelihood calculation)

   <blockquote>
vis_sample allows you to sample visibilities from a user-supplied sky-brightness image.
    
   * (u,v) grid points can either be supplied by the user, or can be retrieved from a template uvfits file / measurement set.
        * The results can be output either to a uvfits file or returned back to the user (for scripting)
   </blockquote>

* uvfits file has channel frequencies (what units?)

* converts to LSRK velocities in m/s by using $v = c \ (1-f_{\rm{native}} \ / \ f_{\rm{rest}})$ (rest frequency of 12CO J=2-1 in [Hz])

* find channels that correspond to ±500 m/s

Code makes a **deep copy** of the data from `import_data_uvfits` method
* shallow copies still refer to original object (more dependent)
	* if you make changes to the original object, the shallow copy will also have changes

* deep copies are recursive, and link to themselves (more independent)


__On Convolution and Windows:__
* Convolution is $(f \star g)(t) = \int_{-\infty}^{\infty}f(\tau)*g(t-\tau)d\tau$
	* describes how the shape of one function is modified by another
	* basically multiplication and then integration

* Window functions are a kind of tapering mechanism (non-constant scaling by a given tapering function, higher order than a scale)
	* mathematical function that is symmetric around middle of the interval (bell shaped), and zero-valued outside of some chosen interval 
	* allows better detection of transient events/time-averaging of frequency spectra

* signals from interferometer are convoluted with window functions to _aa_ time-averaging of frequency spectra when taking the Fourier Transform




**Questions:**
* assigning object names to variables? (dvis_native.VV seems like it's self referential)
* what is VV?
	* has shape (123, 27090)
	* visibilities per channel (123 channels) with v?





# 10/1 Notes

## Process
UVfits file is fake data with frequencies
1. process it so that it reads in terms of velocities, reads in correct coordinates, and identifies the min-max boundaries for fake data
2. calculate covariance matrix for chi-squared analysis later
3. load template model frequencies

4. __likelihood calculation__
    * generate model cube visibilities according to inputted parameters/default values from passer
    * sample fourier transforms of visibilities onto u, v points
    * convolute FT of visibilities? (frequency-domain) with FT of autocorrelation of the Hann window function (also called __spectral response function (SRF)__, frequency-domain)
        * SRF makes things blurry, because the FT of the window function needs to be sampled for short time?
    * interpolation of channel maps
        * figure out LSRK frequencies that are relevant, and interpolate frequencies from data to frequencies from model 
    * in current version, all of this is done for only the middle channel map of the observation (~450 seconds)

* **but repeat #4 for every timestamp (for 900 second observation, averaged every 30 seconds, so 30x)**

## Misc
* UVfits files have a 4D array
    * for each separate u & v coordinate
    * first two indices [0, 1] are real and imaginary visibilities
    * the second index is the weights (1/variance of each file)
    * third index is polarization
        * left/right or x-y polarizations?
        * treat these as independent measurements bc they're usually related to measuring device (interferometer)

* where to get constant values from (e.g. 12CO J=2-1 transition)
    * NASA JPL has a database called ['LAMBDA'](https://lambda.gsfc.nasa.gov/product/planck/curr/planck_prod_esa.cfm)
* topocentric- Earth reference frame
* the data/model/covariance matrix must be binned by a factor of $\geq$ 2
    * covariance matrix is non-invertible if it is not
        * has a high condition number, which dictates how invertible it is
        * if there is no convolution, then there is a high amount of noise that is introduced into log likelihood
    * __what this means:__ things must be binned because channels are intrinsically blurred by SRF
        * spectra from channel maps are not independent, they are dependent on their i+1 counterparts
    * if there are 100 frequencies, then binning by 2 would result in 50 frequencies
    * binning is a maximization problem, you want to bin as little as possible, but the more you bin, the more independent the channels become
        * more bins, less covariance/more independence

## Further Questions
* We have more spectral information in LSRK frame? Why is that?
* channels become blurred by SRF, but are they independent otherwise?

* dvis_native.freqs = frequency for each channel (123 values), in terms of rest frequency frame
* vlsrk_native = velocity for each channel, converted to LSRK velocities [m/s] (123 values)

# 10/5 Notes

* __On LSRK__
    * LSRK frame is supposed to represent fixed reference frame wrt Earth and disk
        * only thing to account for is projected velocities?
    * We have more information for LSRK velocities because of overlapping frequencies
        * ALMA records data into fixed topocentric channels, but LSRK frequencies change from channel to channel
            * Each channel has ~30 LSRK velocities, and the LSRK velocities change with time
            * Converting from TOPO to LSRK requires regridding:
                * Actual transformation (geometric equation)
                * Taking 100 channels and 30 integrations (~3,000 LSRK velocities) and interpolating them with the frequencies that we want (TOPO converted LSRK)
                * Finding desired TOPO frequencies and finding interpolated LSRK velocities for each channel
* __On Data Treatment__
    * model that generates cube is oversampled because of the convolution
        * supposed to mimic 'infinite' signal stream that is being fed into correlator
        * theory is that it leads to a more realistic/accurate convolution product
    * SRF function (Hann vs. simplified)
        * SRF function is what causes covariance from channel to channel
            * Visibilities are not something that can be easily broken down/taken out/be independent (only in theory, but not in practice)
                * Correlator automatically correlates visibilities with SRF
        * visibilities don't have gradients from channel to channel?, which is why we can use simplified SRF
            * simplified SRF is essentially delta functions at 0.25, 0.5, and 0.75
                * the FWHM of the Hann SRF function is 2 channels (spectral resolution)
        
    * Log-likelihood for known quantities should render a $\chi^2$ value of 0
        * except there's a fraction of 0.71 that might be due to convolution steps/oversampling
        * but this is really insignificant, because it's 0.71 of the data out of 1 million
* On molecular compounds
    * $^{12}\rm{CO}$ is normal carbon monoxide
    * $\rm{H}^2$ is hydrogen gas (most abundant)
    * We don't use $\rm{H}^2$ for multiple reasons:
        * it is pure, does not have rotational transition states due to how symmetric of a molecule it is
        * no magnetic moment, has quadropole terms in its moments that are far weaker than monopole/dipole
    * Use $^{12}\rm{CO}$ for multiple reasons:
        * Second most abundant element in disks
        * Is optically thick
            * Has lots of emission from only a little bit of gas
        * Bright even at small radii
            * easier to reconstruct for velocity measurements/dynamical masses
        * Specifically J=2-1 is useful because Earth's atmosphere is most transparent with 2-1
            * 1-0 is too hard to observe, is too close to oxygen spectar that is present in Earth's atmosphere
            * 3-2 is also used, but (i forget why not)
    * We say $^{12}\rm{CO}$ becuase it is an isotopologue of carbon
* On larger scale project:
    * We're hoping that post-processing of log-likelihood has a better result than typical practices that don't do this
    * Need to stress test model for different scenarios in which to use the model/
    * Reduction of data by time-averaging
        * help advocate for this way of thinking (exploration), and also helps people make better inferences (inferencing)
    * `dynesty` might be better than `emcee` for calculating posteriors (in a more mathematical way)

## Steps you took to build an interpolator
1. Figured out mechanism behind 1D linear interpolation (literally just $y = mx + b$)
2. Built basic version of it
3. Tested a dataset that you can control/easily replicate by hand and compared it to other interpolator (`scipy.interpolate.interp1d`)
4. Work out any kinks, because your code will not be perfect
5. Made functions Jit-able

**To do**

6. Scale up to real dataset, and make sure output is correct
7. Error handling, and making sure conditions are met before interpolation starts

# Numba Notes

* For a function that takes scalars as input: use decorator `@numba.vectorize` [[source]](https://numba.pydata.org/numba-doc/dev/user/vectorize.html?highlight=vectorize)
    * For a function that takes arrays as input (ufunc?): use decorator `@numba.guvectorize`
        * Syntax for writing signatures/data types:
            * ```@guvectorize(["void(f8[:], f8[:], c16[:,:], c16[:,:])"], "(DVIS_freq), (LSRK_freq), (b, q), (r, s)", nopython=True)```
            * arrays are denoted as \<datatype>\[\<dimensions>\]
                * ex. `c16[:,:]` = 2D array with complex 64-bit digits
            * the layout (symbols) for scalars are just `()`
            * `void()` means that the function doesn't return anything, otherwise it would be whatever datatype
                * useful for `guvectorize`, where you can't make a function return something
                    * on this note, can only write data to output array by indexing for `guvectorize`
* `@overload` is useful if you have a function that you want to replicate as an unsupported function, and you want Numba to screen the data types of the unsupported function and optimize your jit function [[source]](https://numba.pydata.org/numba-doc/dev/extending/overloading-guide.html?highlight=overload)
* `nopython=True` is what does all the JIT compilation for Numba, otherwise it will run in Python compiler (instead of machine-level compiler)
* `@njit` makes a function jit-able, with the **n** as shorthand for the `nopython=True` argument

# 10/8 Notes

* Next steps: run many different inferences on model, see how outside inferences impact the fit of the model
    * But we also want to know what the intrinsic biases of the model are, so that people can make good inferences
    * By running no noise into the model, we should be getting back the input parameters that we put in
    * By running noise into the model, we find out what the biases of the model are
        * Noise is likely to bias the widths (errors) of the parameters, but can also bias the peaks (mean value for distribution)
        * We are unsure what the noise should look like (noise is approximated to be gaussian, a la central limit theorem)
            * Just a gaussian dist. applied to the real and imaginary visibilities
            * In theory, we could run tests on the noise to see how the noise is shaped, but takes too long?


* My next steps:
    * Modeling corrupted data (via a sampling method, either `emcee` or `dynesty`)
        * need to see which one to use first before actually modeling
    * Seeing how `dynesty` works for our data
        * MCMC (`emcee`) has trouble working with many dimensions (parameters)
            * In our case, we have 13-14 mostly independent parametesr
            * Can sometimes get stuck in parameter space/all parameters are largely independent from one another, and `emcee` usually works better with more correlated parameters?
                * need a way to separate posterior sampling
            * `dynesty` uses __nested sampling__ to calculate the convergence of posterior probabilities in parameter space
                * does not use random walks, it randomly chooses parameters
                    * creates a nest around probable regions, and rejects values outside that 'nest'
                * also can calculate priors (evidence) unlike `emcee`
            * coding-wise, the documentation/programming is more verbose than `emcee`
        * eventually will run this with fake noiseless visibilities to see how it compares to `emcee`
            * need to figure out how many points to sample from, if burn-in is required, etc.
        

## Notes on `dynesty`

* Dynesty uses **nested sampling** to calculate posterior probability distributions according to Bayesian statistics
    * More on __nested sampling__:
        * Randomly samples parameter space according to Bayesian __prior__ (defined in a prior function)
            * __Prior__ tells us how the parameters are distributed (i.e Gaussian, uniform, etc.)?
            * creates bounds based on particular shape (multiple ellipsoids, cubes, spheres, etc.) and draws samples 'live points' (coordinates within prior parameter space)
            * Figures out which point has the least likelihood, discards it, and samples from prior again
            * By doing this, the algorithm creates a nested shell within prior parameter space where the parameters most likely are
            * stops running based on # of points, or other bounding conditions
        * Priors must integrate to 1 to be considered 'proper'
    * Also provides way to estimate the __Bayesian evidence__ of a posterior
        * > The evidence is entirely dependent on the “size” of the prior... using less “informative” priors will increase the expected number of nested sampling iterations


## Project Timeline
---
* __Week 11:__ Fundamentals of Statistical Inferencing
    * Goal: learn how to use `emcee` and more about MCMC
    * _Frustrated about how MCMC works/implementing Metropolis-Hastings_
* __Week 12:__ Fake data run through `emcee` and interferometry primer
    * Goal: Learn more about interferometry
    * Clean up analysis script/work on visualizations
    * _Frustrated on how difficult interferometry is_
* __Week 13:__ Understand `logL_test.py` and its workings
    * Goal: Also learn how to use Numba
    * _Frustrated on not knowing what your role in the project is_
* __Week 14:__ Numba speed-up
    * Goal: Learn how to use Numba
    * Goal: Experiment with how we can speed up interpolation
    * Completed (10/8): Basic 1D linear interpolator (with simple data set)
    * Completed (10/9): Got Numba to run 4x faster than SciPy interpolator (but not too accurate yet)
    Feel like I know how to use Numba now!
* __Week 15:__ `dynesty` learning
    * Goal: Learn how to use `dynesty`
    * Goal: Finish interpolator and make it accurate